# 科研构思
该文本将记录我的主要科研方案与计划
## 核心思想
寻找一套全新通用的灵巧手动作空间表征，替换单纯的关节位形空间。该套表征可更好地反映灵巧手的实际操作能力，从而提升灵巧手在各类复杂任务中的表现。
<br>
与其费尽心采取各种措施教手去完成各种复杂动作，不如在更合适的动作表征空间里进行学习。技能是手在学习过程中自己涌现的，而不是人为教出的。
## 期望优势
我希望我的提出的新表征将拥有怎样的优势，这将为基准实验、消融对比实验的设计提供指引
* **具身泛化性**：新的动作表征空间能适用于所有灵巧手模型，学出的策略也能泛化到不同手型
* **任务无关性**：新表征统一适用于各类下游场景任务，无需为每个任务单独设计动作空间
* **技能通用性**：新表征可统一如抓取、旋转、重定向、手内轨迹跟踪等多种技能的学习
* **数据高效性**：灵巧手能更高效地学习各类技能，增大样本效率
* **物理一致性**：学习出的动作天然流畅平滑，符合物理规律
## 关键灵感
一是从位形空间转向**任务空间**。位形空间最易学，但与机器人运动学结构紧密耦合，无法泛化到不同的构型。任务空间适合密集接触型的手物交互，任务与物理语义更清晰，且与具体构型无关，更易于跨构型迁移。
<br>
二是**任务空间上的表征与学习的内容**。查阅了大量文献与研究，如《A Primer on SO(3) Action Representations in Deep Reinforcement Learning》，发现几点关于SO(3)表征的关键结论，我这里迁移到SE(3)重做表述：
- 齐次变换矩阵 $T$ 与 $SE(3)$ 同构，但直接预测 $T\in SE(3)$ 并不被推荐，主要是 $T$ 几何上属于微分流行的特殊正交群，属于非欧空间，神经网络不好直接预测，而需要投影
- 更进一步说，$T=(R_{1:2}, p)$ 由于其光滑唯一的 $SE$ 表示性和相对小的冗余度，更适合监督学习任务，但对DRL表现并不理想 
- 相对增量动作相比全局动作，可独立于全局坐标系，有助于策略泛化。且相对动作对于策略探索表现更佳
- 李代数表征的旋量（如 $\mathcal{V} \in se(3)$）相比其他表征（如四元数、欧拉角等），在DRL中表现更好
- 更具体的，旋量所在的局部切空间增量动作，在动作缩放后的小步长（避免奇点影响）下训练效果更好
- 需要注意熵正则化对动作探索的影响，该问题源于动作投影扭曲了随机分布，稀疏奖励会放大包括奇异、熵正则化的影响，而密集奖励则相对缓解此类问题
## 表征提议
这块我将具体提出自己选取的动作表征形式，并如何适配于RL中的动作空间
### 表征形式
**表征量**
<br>选取 $\mathcal{V}_b = [\omega_b^T, v_b^T]^T\in se(3)$，表示手指末端的刚体坐标系 $\{b\}$ 的原点在 $\{b\}$ 下的瞬时速度。
<br>旋量统一了旋转和平移的表示，且其线性空间性质使得神经网络更易于预测。它描述的是整个刚体的瞬时运动状态，而非某个点。
其反对称矩阵获取方式如下：
$$T_{}^{-1}\dot{T}=\begin{bmatrix}R^{T} & -R^{T}p\\ 0 & 1\end{bmatrix}\begin{bmatrix}\dot{R} & \dot{p}\\ 0 & 0\end{bmatrix}=\begin{bmatrix}R^{T}\dot{R} & R^{T}\dot{p}\\ 0 & 0\end{bmatrix}=\begin{bmatrix}\left\lbrack\omega_{b}\right\rbrack & v_{b}\\ 0 & 0\end{bmatrix}=\left\lbrack\mathcal{V}\right\rbrack\in se\left(3\right)$$
然后提取 $\omega_b$ 和 $v_b$ 即可。

**学习过程**
<br>在RL的流程中，先接收观察 $o_t$，然后输入给策略 $\pi(a_t|o_t)$，输出的动作量 $a_t$ 即为 $\mathcal{V}_b$，然后发送给环境，得到下一个状态 $o_{t+1}$ 和奖励 $r_t$，构成了一个交互回合 $(o_t, a_t, r_t, o_{t+1})$。
![image-2.png](attachment:image-2.png)
### 动作设计
#### 动作形式
关于 $\mathcal{V}_b$ 具体怎么作用于环境，有许多不同的形式。
<br>从坐标体系上，$\mathcal{V}_b$ 都是**相对**的，这避免了全局坐标系的影响，更有利于泛化。而 $\mathcal{V}_s$ 则与 $\{s\}$ 的选取有关，泛化性较差。
<br>第一种作用形式，是 $$T_{sb}\left(t+1\right)=T_{sb}\left(t\right)\exp\left(\left\lbrack\mathcal{V_{b}(t)}\right\rbrack\Delta t\right)$$
先将学到的旋量 $\mathcal{V}_b$ 通过指数投影为齐次变换矩阵增量，然后作用于当前的末端位姿 $T_{sb}(t)$，得到下一个时刻的末端位姿 $T_{sb}(t+1)$。然后由逆运动学求解出对应的关节位形 $\theta(t+1)$，作为环境的实际执行动作。
<br>第二种作用形式，为$$\theta\left(t+1\right)=\theta(t)+J_{b}^{+}\left(\theta\left(t\right)\right)\mathcal{V_{b}}\left(t\right)\Delta{t}$$
该形式来源于速度逆运动学与 **Newton-Raphson 方法**的迭代公式。这里无需通过**指数映射投影**，而是直接将旋量 $\mathcal{V}_b$ 通过雅可比矩阵的伪逆 $J_b^+$ 映射为关节空间的增量 $\Delta\theta(t+1)$，然后作用于当前关节位形 $\theta(t)$，得到下一个时刻的关节位形 $\theta(t+1)$，作为执行动作。这种形式更直接地将任务空间的动作映射到关节空间，而无需经过逆运动学求解过程。
<br>第二种形式有不同的变体：
* **DLS变体**：$$J_{dls}^{\dagger} = J^T (JJ^T + \lambda^2 I)^{-1}$$
  $$\theta\left(t+1\right)=\theta(t)+J_{dls}^{+}\left(\theta\left(t\right)\right)\mathcal{V_{b}}\left(t\right)\Delta{t}$$
    该变体通过阻尼最小二乘法（Damped Least Squares）计算雅可比伪逆，增强了在奇异位形附近的稳定性，避免了数值不稳定问题。
> 补充：任何矩阵都能进行奇异值分解，对雅可比矩阵进行奇异值分解，则其角度增量可表示为$$J_{b}=U\Sigma V^{T},\quad \Delta\theta=J_{b}^{+}\mathcal{V_{b}}\Delta t=\left(\sum\nolimits_{i}\frac{1}{\sigma_{i}}v_{i}u_{i}^{T}\right)\mathcal{V_{b}}\Delta t$$
> 若位于奇异位形，将导致部分奇异值 $\sigma_i$ 接近于0，从而使得 $\frac{1}{\sigma_i}$ 变得非常大，进而导致角度增量 $\Delta\theta$ 过大，影响学习稳定性。DLS方法通过引入阻尼因子 $\lambda$，将奇异值调整为 $\sigma_i^2 + \lambda^2$，从而避免了除以接近零的数值，提高了数值稳定性。如下
> $$\Delta\theta = J_{dls}^{+}\mathcal{V_{b}}\Delta t = \sum\nolimits_{i} \frac{\sigma_i}{\sigma_i^2 + \lambda^2} v_i u_i^T \mathcal{V}_b \Delta t$$
> DLS可以从优化的角度理解为在最小化位置误差的同时，加入了对关节速度的惩罚项，从而平衡了精度和稳定性。公式为$$\min_{\Delta\theta} \| J_b \Delta\theta - \mathcal{V}_b \Delta t \|^2 + \lambda^2 \| \Delta\theta \|^2$$

* **PD控制变体**：在仿真和实际中，并不是直接将 $\Delta\theta(t+1)$ 作为执行动作，而是作为目标位置，结合PD控制器来计算实际的**关节力矩** $\tau$，这个力矩才是真正施加给底层控制器的命令。
\begin{gather}
\theta_{actual}\left(t\right)=\theta(t)\\ \dot{\theta}_{target}(t)=J_{b}^{+}\left(t\right)\mathcal{V}_{b}(t)\\ \theta_{target}\left(t+1\right)=\theta_{actual}(t)+\dot{\theta}_{target}(t)\Delta t\\ \tau=K_{p}\left(\theta_{target}\left(t+1\right)-\theta_{actual}\left(t\right)\right)+K_{d}\left(\dot{\theta}_{target}(t)-\dot{\theta}_{actual}\left(t\right)\right)
\end{gather}
该变体的合理性存疑，涉及时间系统约定，后续有待补充。后经过确认，该公式基本正确
* **位置误差补偿PD变体**：  
  该变体在速度逆运动学的基础上，加入了末端位姿误差的补偿项。  
  其中 $Error_{pose}$ 表示末端当前实际位姿与目标位姿之间的误差，通常可用SE(3)对数映射（李代数）表示：
  $$
  Error_{pose} = \log\left(T_{sb}^{target} (T_{sb}^{actual})^{-1}\right)^\vee
  $$
  其中 $\log(\cdot)^\vee$ 表示SE(3)的对数映射并取向量形式。  
  最终控制律为：
  $$
  \dot{\theta}_{cmd} = J^+ \left(\mathcal{V}_b + K_{error} \cdot Error_{pose}\right)
  $$
该变体必要性存疑，如果使用PD控制变体，已经携带了位置反馈，这里还需要吗？
#### 范围取舍
由于旋量 $\mathcal{V}_b$ 理论上无界，因此需要对其进行范围限制，以适应实际的机器人手指运动能力。
<br>首先 $\mathcal{V}_{b}=\begin{bmatrix}\omega_{b}\\ v_{b}\end{bmatrix}$，范围限制可分为对 $\omega_b$ 和 $v_b$ 两部分。

**角速度**
<br>对于角速度 $\omega_b$，有 $$\omega_{b}=\hat{\omega}\left|\right|\omega||=\hat{\omega}\frac{\Delta{\theta}}{\Delta{t}}=\hat{\omega}\frac{\pi}{k}$$这里是通过螺旋轴的旋转角来估算角速度大小，等价于 $\{b\}$ 每秒围绕绕螺旋轴转的最大度数 $|\frac{\pi}{k}|$。因为李代数旋量在 $\pi$ 处奇异，故每个时间步的旋转角 $\Delta{\theta}=\frac{\pi}{k}\Delta{t}$ 不能过大，$k$ 超参数便是调控该幅度范围的。

**线速度**
<br>对于线速度 $v_b$，有 $v_{b}=R_{}\dot{p}$。$v_b$ 是 $\{b\}$ 原点在 $\{b\}$ 下的线速度，因为 $\{b\}$ 一直在变化，故稍显不直观。而 $\dot{p}$ 则是 $\{b\}$ 原点在 $\{s\}$ 下的线速度，更直观。利用矩阵范数与向量范数的相容性，有
$$||v_{b}||_{\infty}=||R_{}\dot{p}||_{\infty}\leq\sqrt{3}||R_{}||_2||\dot{p}||_{\infty}=\sqrt{3}\sqrt{\lambda_{max}\left(R^{T}R\right)}\left|\right|\dot{p}\left|\right|_{\infty}=\sqrt{3}\left|\right|\dot{p}\left|\right|_{\infty}$$
因为旋转矩阵 $R$ 的2-范数为1，故可通过预估 $\dot{p}$ 的大小来间接设计 $v_b$ 的范围
## 理论补充
#### 范围取舍
由于旋量 $\mathcal{V}_b$ 理论上无界，因此需要对其进行范围限制，以适应实际的机器人手指运动能力。
<br>首先 $\mathcal{V}_{b}=\begin{bmatrix}\omega_{b}\\ v_{b}\end{bmatrix}$，范围限制可分为对 $\omega_b$ 和 $v_b$ 两部分。

**角速度**
<br>对于角速度 $\omega_b$，有 $$\omega_{b}=\hat{\omega}\left|\right|\omega||=\hat{\omega}\frac{\Delta{\theta}}{\Delta{t}}=\hat{\omega}\frac{\pi}{k}$$这里是通过螺旋轴的旋转角来估算角速度大小，等价于 $\{b\}$ 每秒围绕绕螺旋轴转的最大度数 $|\frac{\pi}{k}|$。因为李代数旋量在 $\pi$ 处奇异，故每个时间步的旋转角 $\Delta{\theta}=\frac{\pi}{k}\Delta{t}$ 不能过大，$k$ 超参数便是调控该幅度范围的。

**线速度**
<br>对于线速度 $v_b$，有 $v_{b}=R_{}\dot{p}$。$v_b$ 是 $\{b\}$ 原点在 $\{b\}$ 下的线速度，因为 $\{b\}$ 一直在变化，故稍显不直观。而 $\dot{p}$ 则是 $\{b\}$ 原点在 $\{s\}$ 下的线速度，更直观。利用矩阵范数与向量范数的相容性，有
$$||v_{b}||_{\infty}=||R_{}\dot{p}||_{\infty}\leq\sqrt
### 李括号与Chow定理

**Chow定理**（Chow-Rashevskii Theorem）是控制理论和次黎曼几何中的核心定理，对于灵巧手等欠驱动机器人系统的可控性分析至关重要。

它直接回答了灵巧手等欠驱动系统的关键问题：
> **如果我的控制输入（如手指关节数）少于系统的自由度（如末端的SE(3)空间），是否仍能实现任意操作？**

Chow定理的数学核心是**李括号（Lie Bracket）**。它说明：只要通过李括号生成的运动方向足够多，能"填满"整个操作空间，灵巧手就理论上可以实现任意操作。

#### 1. 灵巧手的欠驱动建模

以灵巧手为例，假设我们关注手指末端位姿的状态 $\mathcal{V} \in se(3)$（6维旋量空间），但实际控制输入是关节速度 $\dot{\theta} \in \mathbb{R}^m$，其中 $m$ 为灵巧手的关节数。系统运动学可写为：
$$
\mathcal{V} = J(\theta) \dot{\theta}
$$
其中：
* $\mathcal{V}$：末端刚体坐标系 $\{b\}$ 的旋量（6维，包含角速度和线速度）
* $\dot{\theta}$：关节速度向量（$m$ 维）
* $J(\theta)$：雅可比矩阵（$6 \times m$ 维）

**关键点：** 对于灵巧手，$m < 6$（例如单根手指通常只有4个关节）。每个关节直接产生的速度约束来自雅可比矩阵的各列 $g_i = J_{:,i}$，无法直接生成所有6个自由度的运动。

#### 2. 李代数秩条件 (LARC) 与灵巧手可控性

尽管单个关节的直接贡献受限，灵巧手仍可通过**交替驱动**不同关节产生复杂运动。判断灵巧手是否能实现复杂操作，需考察"运动生成能力"：

1. **直接运动：** 通过每个关节直接驱动的速度方向，对应雅可比矩阵的各列：
$$g_i = J_{:,i}(\theta), \quad i=1,\ldots,m$$

2. **间接运动（李括号）：** 交替驱动不同关节，使得关节空间的非交换性产生额外的速度方向。李括号定义为：
$$[g_i, g_j] = \frac{\partial g_j}{\partial \theta} g_i - \frac{\partial g_i}{\partial \theta} g_j$$
它表示在速度空间中，先沿 $g_i$ 方向、后沿 $g_j$ 方向、再反向 $g_i$ 方向、最后反向 $g_j$ 方向这样一个"无穷小闭合回路"所产生的速度增量。

3. **高阶运动：** 继续对已有方向做李括号，如 $[g_k, [g_i, g_j]]$，获得更复杂的运动方向。

所有这些方向张成的空间称为**李代数** $\mathcal{L}(\theta)$：
$$
\mathcal{L}(\theta) = \text{span} \left\{ g_i,\ [g_i, g_j],\ [g_k, [g_i, g_j]],\ \ldots \right\}
$$

#### Chow定理在灵巧手中的表述
> 若在某配置 $\theta$，由关节驱动产生的李代数 $\mathcal{L}(\theta)$ 的秩等于末端操作空间维度（对SE(3)为6），即
> $$
> \text{rank}(\mathcal{L}(\theta)) = 6
> $$
> 则灵巧手系统在该配置**小时间局部可控**（STLC），即理论上能通过有限步关节运动到达末端位姿的任意邻域。若对所有配置都成立，则全局可控。

这就是**李代数秩条件（LARC）**，是灵巧手设计与控制的理论基础。

#### 3. 对旋量表征空间的合理性解释

Chow定理直接为我们的**旋量表征空间** $\mathcal{V}_b \in se(3)$ 提供了理论支撑：

- **表征必要性：** 虽然灵巧手的关节数 $m < 6$，但通过李括号的高阶组合，可在操作空间中生成完整的6维旋量方向。因此，用6维旋量 $\mathcal{V}_b$ 作为策略的输出动作空间是合理的，它能够编码灵巧手所有理论可达的末端运动。

- **控制闭环：** 在执行时，旋量 $\mathcal{V}_b$ 通过雅可比矩阵映射回关节空间：
$$
\dot{\theta} = J^{+}(\theta) \mathcal{V}_b
$$
其中 $J^{+}$ 是雅可比伪逆。由于李括号保证了操作空间的完全可控性，这个映射虽然在局部看似有约束（$J$ 的秩 $\leq m < 6$），但全局上能够实现对所有 $\mathcal{V}_b$ 方向的探索和学习。

- **学习优势：** 策略直接在完整的SE(3)操作空间中学习，而不受关节数量的限制。这使得学到的技能具有更强的**任务语义**（如"旋转物体"、"平移物体"）和**跨构型泛化性**（不同关节配置下相同的旋量命令对应相似的物理效果）。

#### 4. 总结

Chow定理的深层意义：**李括号让灵巧手突破输入数目的限制。** 即使单个关节数远少于末端的6个自由度，只要李括号生成的运动方向足够多，灵巧手就能在操作空间中实现完整的可控性。

在我们的控制框架中：
1. **旋量 $\mathcal{V}_b$** 作为策略输出，编码末端的完整运动意图。
2. **雅可比矩阵 $J(\theta)$** 承载了系统的欠驱动约束，但通过李括号的非线性效应补偿。
3. **RL策略学习** 可以自然地在这个完整的6维操作空间中进行探索，无需显式处理欠驱动的复杂性——这由控制底层的李括号效应自动满足。

这也解释了为什么**旋量表征** $\mathcal{V}_b$ 相比关节空间表征更具优势：它捕捉了灵巧手控制的本质——在受限的关节输入下，通过复杂的非线性动力学与几何结构，实现丰富的操作能力。
{3}||R_{}||_2||\dot{p}||_{\infty}=\sqrt{3}\sqrt{\lambda_{max}\left(R^{T}R\right)}\left|\right|\dot{p}\left|\right|_{\infty}=\sqrt{3}\left|\right|\dot{p}\left|\right|_{\infty}$$
因为旋转矩阵 $R$ 的2-范数为1，故可通过预估 $\dot{p}$ 的大小来间接设计 $v_b$ 的范围
1. 关于具身泛化，它可能与末端具体位置及坐标系约定有关，例如，手指末端坐标系 $\{b\}$ 的轴方向，是否与手指的主要运动方向对齐？是否与物体的参考系有关？这些都会影响 $\mathcal{V}_b$ 的表达能力和泛化性。如何避免或解决该问题呢？
<br>一个解决方案是**虚拟规范化坐标系** (Virtual Canonical Frame)，在工程上做一个**语义对齐**，规定 $b_{can}$ 的语义：
*   原点：位于指尖几何中心或接触面中心。
*   Z 轴：始终指向指尖法线方向（Outward Normal）。主要表示手指的伸缩。
*   Y 轴：始终指向指尖侧面（例如指甲盖切线方向）。主要表示手指的摆动。
*   X 轴：由右手定则确定。一般是指腹法线方向。主要表示手指的捏合。

策略 $\pi$ 输出的动作是相对于这个统一坐标系 $\{b_{can}\}$ 的旋量 $\mathcal{V}_{can}$。

在执行时，我们需要将其变换回实际末端刚体 $\{b_{raw}\}$ 才能通过 Jacobian 求解，因为指尖位置的末端坐标是虚拟设置的Xform，不是刚体，无法获取雅可比矩阵。

利用**伴随变换 (Adjoint Map)**：
$$
\mathcal{V}_{raw} = \text{Ad}_{T_{raw \to can}} (\mathcal{V}_{can})
$$
其中 $T_{raw \to can}$ 是一个固定的静态变换矩阵（通常只是一个旋转矩阵，用于对齐轴向）。

**修正后的控制律：**
$$
\theta(t+1) = \theta(t) + J_{raw}^{+}(\theta) \cdot \underbrace{\text{Ad}_{T_{raw \to can}} (\mathcal{V}_{policy})}_{\text{Mapped to raw frame}} \cdot \Delta t
$$
## 任务设计
1. 关于具身泛化，它可能与末端具体位置及坐标系约定有关，例如，手指末端坐标系 $\{b\}$ 的轴方向，是否与手指的主要运动方向对齐？是否与物体的参考系有关？这些都会影响 $\mathcal{V}_b$ 的表达能力和泛化性。如何避免或解决该问题呢？
<br>一个解决方案是**虚拟规范化坐标系** (Virtual Canonical Frame)，在工程上做一个**语义对齐**，规定 $b_{can}$ 的语义：
*   原点：位于指尖几何中心或接触面中心。
*   Z 轴：始终指向指尖法线方向（Outward Normal）。主要表示手指的伸缩。
*   Y 轴：始终指向指尖侧面（例如指甲盖切线方向）。主要表示手指的摆动。
*   X 轴：由右手定则确定。一般是指腹法线方向。主要表示手指的捏合。

策略 $\pi$ 输出的动作是相对于这个统一坐标系 $\{b_{can}\}$ 的旋量 $\mathcal{V}_{can}$。

在执行时，我们需要将其变换回实际末端刚体 $\{b_{raw}\}$ 才能通过 Jacobian 求解，因为指尖位置的末端坐标是虚拟设置的Xform，不是刚体，无法获取雅可比矩阵。

利用**伴随变换 (Adjoint Map)**：
$$
\mathcal{V}_{raw} = \text{Ad}_{T_{raw \to can}} (\mathcal{V}_{can})
$$
其中 $T_{raw \to can}$ 是一个固定的静态变换矩阵（通常只是一个旋转矩阵，用于对齐轴向）。

**修正后的控制律：**
$$
\t
##
#### 手内精细操作
* 
### 评判指标
- 通用指标有`成功率、效率（steps/time）、sample efficiency、鲁棒性、泛化性`
- baselines算法有PPO、SAC、TD3等
- ablations有  
1. Vision‑only（RGBD + proprio） vs Vision+simulated tactile（仿真 contact forces）。
2. With / Without ADR（域随机化）— 看 sim→real 成本。
3. Action type ablation：Position target vs Velocity vs Torque**重定向任务**
<br>这个可结合IsaacLab现有环境：Isaac-Repose-Cube-Allegro-v0，还可改版为Leaphand手

* **手内旋转任务**
<b
### 高难度任务【后续可考虑】r>这个可结合我/Leaphand官方的手内旋转任务环境

* **轨迹跟踪任务**
<br>可以新设有关

测评指标：末端物体位姿误差、成功率、平均完成步数# 基准任务
可s
#### 铰链对象操作
面向旋转阀门、开关、拿握使用剪刀、抽屉操作等，主要评估成功率、对未见对象（物理参数变化）的泛化能力
im2real的可行场景任务heta(t+1) = \theta(t) + J_{raw}^{+}(\theta) \cdot \underbrace{\text{Ad}_{T_{raw \to can}} (\mathcal{V}_{policy})}_{\text{Mapped to raw frame}} \cdot \Delta t
$$